In [13]:
def load_input(path: str) -> list[str]:
    with open(path) as infile:
        return infile.read().splitlines()


# Part 1

In [ ]:
from dataclasses import dataclass
EXAMPLE = False
if EXAMPLE:
    puzzle_input = load_input("example.txt")
    X_MAX = 7  # 71
    Y_MAX = 7  # 71
    FALLING_BYTES = 12

else: 
    puzzle_input = load_input("input.txt")
    X_MAX = 71
    Y_MAX = 71
    FALLING_BYTES = 1024




@dataclass(frozen=True)
class Position():
    x: int
    y: int




START = Position(0, 0)
END = Position(X_MAX-1, Y_MAX-1)  # end is always bottom right

memory_space = [["." for _ in range(X_MAX)] for _ in range(Y_MAX)]


def parse_input(input: list[str]):
    memory_list = []
    for s in input:
        coords = s.split(",")
        memory_list.append(Position(int(coords[0]), int(coords[1])))
    return memory_list


def place_obsticales(memory_space: list[list[str]], byte_list: list[Position], no_falling_bytes: int):
    for i in range(no_falling_bytes):
        byte = byte_list[i]
        memory_space[byte.y][byte.x] = "#"
    return memory_space


byte_list = parse_input(puzzle_input)
memory_space = place_obsticales(memory_space, byte_list, FALLING_BYTES)


def h(pos: Position, end: Position) -> int:
    from math import sqrt
    return sqrt((pos.x - end.x)**2 + (pos.y - end.y)**2)


def reconstruct_path(cameFrom, current):
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.insert(0, current)
    return list(set(total_path))


def get_neighbors(memory_space, pos: Position):
    neighbors = []
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    for d in directions:
        new_pos = Position(pos.x + d[0], pos.y + d[1])
        if new_pos.x < 0 or new_pos.x >= X_MAX:
            continue
        if new_pos.y < 0 or new_pos.y >= Y_MAX:
            continue
        if memory_space[new_pos.y][new_pos.x] == "#":
            continue
        neighbors.append(new_pos)
    return neighbors


def init_map_with_default(default_value):
    m = {}
    for y in range(Y_MAX):
        for x in range(X_MAX):
            m[Position(x, y)] = default_value
    return m


def find_path(memory_space, start: Position, end: Position):
    openSet = set([start])
    cameFrom = {}

    gScore = init_map_with_default(float("inf"))
    gScore[start] = 0

    fScore = init_map_with_default(float("inf"))
    fScore[start] = h(start, end)

    while openSet:
        current = min(openSet, key=fScore.get)
        if current == end:
            return reconstruct_path(cameFrom, current)
        openSet.remove(current)

        neighbors = get_neighbors(memory_space, current)
        for n in neighbors:
            # d(currecnt, neighbor) == 1 always!
            tentative_score = gScore[current] + 1
            if tentative_score < gScore[n]:
                cameFrom[n] = current
                gScore[n] = tentative_score
                fScore[n] = tentative_score + h(n, end)
                if n not in openSet:
                    openSet.add(n)

    return False


path = find_path(memory_space, START, END)
display(len(path)-1)

# Part 2

In [ ]:
from tqdm import tqdm

EXAMPLE = False
if EXAMPLE:
    puzzle_input = load_input("example.txt")
    X_MAX = 7  # 71
    Y_MAX = 7  # 71
    FALLING_BYTES = 12

else: 
    puzzle_input = load_input("input.txt")
    X_MAX = 71
    Y_MAX = 71
    FALLING_BYTES = 1024




byte_list = parse_input(puzzle_input)
START = Position(0, 0)
END = Position(X_MAX-1, Y_MAX-1)  # end is always bottom right



def binary_search(memory_space, byte_list: list) -> int:
    idx_l = 0
    idx_r = len(byte_list)-1
    while idx_l <= idx_r:
        idx_m = int(((idx_l+idx_r) / 2))
        memory_space = [["." for _ in range(X_MAX)] for _ in range(Y_MAX)]
        memory_space = place_obsticales(memory_space, byte_list, idx_m)
        path = find_path(memory_space, START, END)
        if path:
            idx_l = idx_m+1
        else:
            idx_r = idx_m-1
    return idx_m
    

def pure_fore():
    for i in tqdm(range(2800,len(byte_list))):
        memory_space = [["." for _ in range(X_MAX)] for _ in range(Y_MAX)]
        memory_space = place_obsticales(memory_space, byte_list, i)
        path = find_path(memory_space, START, END)
        if not path:
            print(path)
            print(byte_list[i])
            break
